Priprema podataka
- dobivanje parova nizova iz poravnanja višestrukih sekvenci (engl. Multiple sequence alignment)
- korištenje podataka ([HIV Sequence Alignments](https://www.hiv.lanl.gov/content/sequence/NEWALIGN/align.html)) za inicijalizaciju matrica u Baum-Welch algoritmu


In [33]:
import itertools
import numpy as np

from collections import Counter
from collections import defaultdict 
from tqdm import tqdm

In [34]:

def read_fasta(file_path):
    sequences = []
    current_sequence = ""
    current_header = ""
    dozvoljeni_znakovi = set(["-", "G", "C", "T", "A"])



    total_lines = sum(1 for line in open(file_path, 'r'))

    with open(file_path, 'r') as file:
        for line in tqdm(file, total=total_lines, desc="Reading FASTA file"):
            line = line.strip()
            if line.startswith('>'):
                if current_header: # dodaj sekvencu
                    #provjeri je li unutra neki znak različit od "-AGTC"
                    
                    neki_znakovi = set(current_sequence) - dozvoljeni_znakovi
                    if (len(neki_znakovi) == 0):
                        sequences.append(current_sequence)
                current_header = line[1:]
                current_sequence = ""
            else:
                current_sequence += line

    # zadnja sekvenca
    if current_header:
        sequences.append(current_sequence)

    return sequences


fasta_file_path = 'HIV1_ALL_2021_genome_DNA.fasta'

sequences_all = read_fasta(fasta_file_path)[:1000]  # radi ograničenih resursa zadržat ćemo 500 sekvenci
sequences = sequences_all[:11]


Reading FASTA file: 100%|██████████| 1515900/1515900 [00:02<00:00, 603037.07it/s]


In [35]:
print(f"Ukupan broj sekvenci: {len(sequences)}")
print(f"Duljina sekvenci: {Counter([len(i) for i in sequences])}")

Ukupan broj sekvenci: 11
Duljina sekvenci: Counter({14937: 11})


In [36]:
#napravi sve parove od poravnatih sekvenci (i (x,y) i (y,x))
pairs = list(itertools.permutations(sequences, 2))
len(pairs)

110

In [37]:
def process_pair(pair):
    
    prvi = ''.join(a for a, b in zip(pair[0], pair[1]) if a != '-' or b != '-' )
    drugi = ''.join(b for a, b in zip(pair[0], pair[1]) if a != '-' or b != '-' )
    
    novi_prvi = ''.join('' if ((prvi[i] == '-' and i > 0 and drugi[i-1] == '-') or (prvi[i] == '-' and i < (len(drugi) - 1) and drugi[i + 1] == '-')) else prvi[i] for i in range(len(prvi)))
    novi_drugi = ''.join('' if ((drugi[i] == '-' and i > 0 and prvi[i-1] == '-') or (drugi[i] == '-' and i < (len(prvi) - 1) and prvi[i + 1] == '-')) else drugi[i] for i in range(len(drugi)))
    
    return novi_prvi, novi_drugi

def get_pair_values(pair):
    return [
        'M' if a != '-' and b != '-' else
        'Ix' if a != '-' and b == '-' else
        'Iy'
        for a, b in zip(pair[0], pair[1])
    ]
    
def izracunaj_matricu_pocetnih_vrijednosti(elementi):
    pi = [
        elementi["M"] / sum(elementi.values()), 
        elementi["Ix"] / sum(elementi.values()), 
        elementi["Iy"] / sum(elementi.values())
    ]
    # ako slucajno bude 0 to treba zamjeniti s malom vrijednošću
    eps = 0.00000001
    pi = [max(vjerojatnost,eps) for vjerojatnost in pi]
    return pi

def normaliziraj_matricu(matrica):
    eps = 0.00000001
    matrica = matrica/ np.sum(matrica, axis=1, keepdims=True)
    return np.where(matrica == 0, eps, matrica)


In [38]:
stanja_set = ['M', 'Ix', 'Iy']
emisija = sorted(["A", "AA", "AC", "AG", "AT", "C", "CA", "CC", "CG", "CT", "G", "GA", "GC", "GG", "GT", "T", "TA", "TC", "TG", "TT"])
prvi_element_brojac = defaultdict(int)

matrica_prijelaza = np.zeros((len(stanja_set), len(stanja_set)))
matrica_emisija = np.zeros((len(stanja_set), len(emisija)))


# for pair in itertools.permutations(sequences, 2):  # ukoliko zelimo izbjeci spremanje svih permutacija sekvenci u memoriju (maknuti varijablu pairs)
with open("parovi.txt", "w") as file:    
    for pair in tqdm(pairs):
        
        processed_pair = process_pair(pair)
        pair_values = get_pair_values(processed_pair)
        
        prvi_element_brojac[pair_values[0]] += 1
        
        for i in range(len(pair_values) - 1):
            trenutno_stanje = stanja_set.index(pair_values[i])
            sljedece_stanje = stanja_set.index(pair_values[i + 1])
            matrica_prijelaza[trenutno_stanje, sljedece_stanje] += 1
            
        svi_parovi_simbola = []
        for i, simbol_stanja in enumerate(pair_values):
            x,y = processed_pair
    
            if x[i] != '-' and y[i] != '-':
                par_simbola = x[i] + y[i]
            else:
                if x[i] == '-':
                    par_simbola = y[i]
                else:
                    par_simbola = x[i]
        
            svi_parovi_simbola.append(par_simbola)
            matrica_emisija[stanja_set.index(simbol_stanja), emisija.index(par_simbola)] += 1

        file.write(str(svi_parovi_simbola))
        file.write("\n")

100%|██████████| 110/110 [00:03<00:00, 29.36it/s]


In [39]:
pi = izracunaj_matricu_pocetnih_vrijednosti(prvi_element_brojac)
pi

[0.12727272727272726, 0.43636363636363634, 0.43636363636363634]

In [40]:
#pi2 = [0.07368336673346694, 0.4631583166332665, 0.4631583166332665]
#pi2

In [41]:
A = normaliziraj_matricu(matrica_prijelaza)
A

array([[9.97797004e-01, 1.10149792e-03, 1.10149792e-03],
       [3.25920591e-02, 9.67407941e-01, 1.00000000e-08],
       [3.25920591e-02, 1.00000000e-08, 9.67407941e-01]])

In [42]:
#A2 = np.array([[9.97843253e-01, 1.07837332e-03, 1.07837332e-03],
#       [4.12591903e-02, 9.58740196e-01, 6.13427300e-07],
#       [4.12591903e-02, 6.13427300e-07, 9.58740196e-01]])
#A2

In [43]:
E = normaliziraj_matricu(matrica_emisija)
E

array([[1.00000000e-08, 3.25694888e-01, 7.07301008e-03, 2.24633936e-02,
        5.07371941e-03, 1.00000000e-08, 7.07301008e-03, 1.53913969e-01,
        2.73699144e-03, 1.24699802e-02, 1.00000000e-08, 2.24633936e-02,
        2.73699144e-03, 2.12802351e-01, 3.52029184e-03, 1.00000000e-08,
        5.07371941e-03, 1.24699802e-02, 3.52029184e-03, 2.00914019e-01],
       [2.77884327e-01, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08,
        1.00000000e-08, 2.15552892e-01, 1.00000000e-08, 1.00000000e-08,
        1.00000000e-08, 1.00000000e-08, 2.76985316e-01, 1.00000000e-08,
        1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 2.29577465e-01,
        1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08],
       [2.77884327e-01, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08,
        1.00000000e-08, 2.15552892e-01, 1.00000000e-08, 1.00000000e-08,
        1.00000000e-08, 1.00000000e-08, 2.76985316e-01, 1.00000000e-08,
        1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 2.2957

In [44]:
#E2 = np.array([[1.00000000e-08, 3.25734397e-01, 8.15964895e-03, 2.39453468e-02,
#        5.72147873e-03, 1.00000000e-08, 8.15964895e-03, 1.50013314e-01,
#        3.06291168e-03, 1.39968051e-02, 1.00000000e-08, 2.39453468e-02,
#        3.06291168e-03, 2.08718026e-01, 3.67871353e-03, 1.00000000e-08,
#        5.72147873e-03, 1.39968051e-02, 3.67871353e-03, 1.98404453e-01],
#       [2.90796102e-01, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08,
#        1.00000000e-08, 2.18200828e-01, 1.00000000e-08, 1.00000000e-08,
#        1.00000000e-08, 1.00000000e-08, 2.74978179e-01, 1.00000000e-08,
#        1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 2.16024890e-01,
#        1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08],
#       [2.90796102e-01, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08,
#        1.00000000e-08, 2.18200828e-01, 1.00000000e-08, 1.00000000e-08,
#        1.00000000e-08, 1.00000000e-08, 2.74978179e-01, 1.00000000e-08,
#        1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 2.16024890e-01,
#        1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08]])
#E2

In [45]:
if __name__ == "__main__":
    with open("datoteka.txt", "w") as file:
        file.write("M\n")
        file.write("20\n")
        file.write("N\n")
        file.write("3\n")
        file.write("STANJA\n")
        for stanje in stanja_set:
            file.write(stanje + " ")
        file.write("\n")
        file.write("SIMBOLI\n")
        for simbol in emisija:
            file.write(simbol + " ")
        file.write("\n")
        
        file.write("<A>\n")
        for red in A:
            file.write(" ".join(map(str, red)) + "\n")

        file.write("<E>\n")        
        for red in E:
            file.write(" ".join(map(str, red)) + "\n")
            
        file.write("<pi>\n")        
        for prob in pi:
            file.write(str(prob) + " ")
            